In [1]:
import os
import json

import pandas as pd
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

import category_encoders as ce

import warnings

warnings.filterwarnings('ignore')

In [2]:
X = pd.read_csv('../data/binned/df.csv')
y = pd.read_csv('../data/binned/y.csv')

with open('../data/binned/interaction_constraints.json', 'r') as file:
    interaction_constraints = json.load(file)

print(interaction_constraints)

for col in X.filter(like='_binned').columns:
    X[col] = X[col].astype('category')

X.filter(like='_binned').info()

[['player_rating_away_player_1', 'player_rating_away_player_2', 'player_rating_away_player_3', 'player_rating_away_player_4', 'player_rating_away_player_5', 'player_rating_away_player_6', 'player_rating_away_player_7', 'player_rating_away_player_8', 'player_rating_away_player_9', 'player_rating_away_player_10', 'player_rating_away_player_11'], ['ewm_away_team_goals', 'ewm_home_team_goals_conceded', 'points_away', 'ewm_shoton_home', 'ewm_shoton_away', 'ewm_possession_home', 'ewm_possession_away', 'points_home_binned', 'home_weighted_wins_binned', 'away_weighted_wins_binned', 'ewm_home_team_goals_binned', 'ewm_away_team_goals_conceded_binned'], ['num_top_players_home', 'num_top_players_away'], ['rating_range_home', 'rating_range_away']]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3040 entries, 0 to 3039
Data columns (total 9 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   avg_home_r

In [9]:
def train_and_evaluate(df, target, preprocessing=None):
    """
    Train an XGBClassifier with specified preprocessing on the training data and evaluate it on the validation data.

    Parameters:
    df (DataFrame): Data
    target (Series): Target data
    random_state (int): Random state for reproducibility
    enable_categorical (bool): Enable categorical feature support
    preprocessing (str): Preprocessing method ('Normalization', 'Standardization', or None)

    Returns:
    float: The F1 score of the model on the validation data
    """
    X_train, X_val, y_train, y_val = train_test_split(df, target, test_size=0.2, random_state=42, stratify=target)

    num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
    cat_cols = X_train.select_dtypes(include=['object', 'category']).columns

    if preprocessing == 'Normalization':
        num_transformer = MinMaxScaler()
    elif preprocessing == 'Standardization':
        num_transformer = StandardScaler()
    else:
        num_transformer = 'passthrough'

    cat_transformer = ce.OneHotEncoder(cols=cat_cols, use_cat_names=True, drop_invariant=True, return_df=True)

    preprocessor = ColumnTransformer([
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ])
    
    X_train_preprocessed = preprocessor.fit_transform(X_train)
    X_val_preprocessed = preprocessor.transform(X_val)

    model = XGBClassifier(random_state=42, enable_categorical=True)
    model.fit(X_train_preprocessed, y_train, eval_set=[(X_train_preprocessed, y_train), (X_val_preprocessed, y_val)], verbose=0, early_stopping_rounds=25)

    y_pred = model.predict(X_val_preprocessed)
    f1_score_val = f1_score(y_val, y_pred, average='weighted')

    return f1_score_val

In [10]:
train_and_evaluate(X, y, preprocessing='Normalization')

KeyboardInterrupt: 

In [ ]:
5322813298157089